# Balanceamento de CLasses

# Analise de Classes Pré Balanceamento

In [0]:
from pyspark.sql import functions as F

try:
    # 1. Carrega a tabela original
    df_original = spark.table("transacoes_db.copper.transacoes")
    
    # 2. Calcula o total de registros
    total_count = df_original.count()
    
    if total_count == 0:
        print("AVISO: A tabela original está vazia. Nenhuma análise para mostrar.")
    else:
        print(f"Total de Registros (Original): {total_count}\n")
        
        # 3. Calcula a distribuição por 'is_fraud'
        print("Distribuição por Classe (Original):")
        df_original.groupBy("is_fraud") \
            .count() \
            .withColumn("percentual", (F.col("count") / total_count) * 100) \
            .orderBy(F.col("count").desc()) \
            .show()

        # 4. Calcula a distribuição detalhada por 'fraud_type'
        print("Distribuição por Tipo de Fraude (Original):")
        df_original.withColumn(
                "tipo_fraude_detalhado", 
                F.when(F.col("is_fraud") == 0, "Legitima").otherwise(F.col("fraud_type"))
            ) \
            .groupBy("tipo_fraude_detalhado") \
            .count() \
            .withColumn("percentual", (F.col("count") / total_count) * 100) \
            .orderBy(F.col("count").desc()) \
            .show()

except Exception as e:
    print(f"ERRO: Não foi possível ler 'transacoes_db.copper.transacoes'. Verifique se a Célula 3 foi executada.")
    print(f"Detalhe: {e}")

## Analise visual Pre Balanceamento

In [0]:
import matplotlib.pyplot as plt

# 1. Carrega a tabela original
df_original = spark.table("transacoes_db.copper.transacoes")

# 2. Calcula a distribuição por 'is_fraud'
df_is_fraud = df_original.groupBy("is_fraud") \
    .count() \
    .orderBy("is_fraud")

# 3. Calcula a distribuição detalhada por 'fraud_type'
df_tipo_fraude = df_original.withColumn(
        "tipo_fraude_detalhado", 
        F.when(F.col("is_fraud") == 0, "Legitima").otherwise(F.col("fraud_type"))
    ) \
    .groupBy("tipo_fraude_detalhado") \
    .count() \
    .orderBy(F.col("count").desc())

# Coleta os dados para visualização
is_fraud_data = df_is_fraud.toPandas()
tipo_fraude_data = df_tipo_fraude.toPandas()

# Gráfico de barras para distribuição por classe
plt.figure(figsize=(6,4))
plt.bar(is_fraud_data['is_fraud'], is_fraud_data['count'], color=['green', 'red'])
plt.xlabel('Classe (is_fraud)')
plt.ylabel('Quantidade')
plt.title('Distribuição por Classe (Pré Balanceamento)')
plt.xticks([0,1], ['Legitima', 'Fraude'])
plt.show()


In [0]:

# Gráfico de barras para distribuição por tipo de fraude
plt.figure(figsize=(10,5))
plt.bar(tipo_fraude_data['tipo_fraude_detalhado'], tipo_fraude_data['count'], color='orange')
plt.xlabel('Tipo de Fraude')
plt.ylabel('Quantidade')
plt.title('Distribuição por Tipo de Fraude (Pré Balanceamento)')
plt.xticks(rotation=45)
plt.show()



In [0]:
# Histograma dos valores transacionados por classe (Pré Balanceamento)
valores_pd = df_original.select("is_fraud", "valor").toPandas()

plt.figure(figsize=(8,5))
for label, color in zip([0,1], ['green', 'red']):
    plt.hist(
        valores_pd[valores_pd['is_fraud']==label]['valor'], 
        bins=30, 
        alpha=0.6, 
        label=f'{"Legitima" if label==0 else "Fraude"}', 
        color=color
    )
plt.title('Distribuição dos Valores Transacionados por Classe (Pré Balanceamento)')
plt.xlabel('Valor da Transação')
plt.ylabel('Frequência')
plt.legend()
plt.tight_layout()
plt.show()

# Balanceamento 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

print("INFO: Iniciando processo de BALANCEAMENTO V11 (Robusto)...")
print("INFO: Classe 1 (Fraude) incluirá a Raiz E os Sintomas (filhos da cadeia).")
print("INFO: Classe 0 (Legítimo) incluirá APENAS transações legítimas.")
print("AVISO: Executando em modo 'SEM CACHE' por instrução. A execução pode ser lenta.")

# Usar a mesma seed global para reprodutibilidade
SEED_BALANCEAMENTO = 42

try:
    df_transacoes = spark.table("transacoes_db.copper.transacoes")

    # =========================================================================
    # PASSO 1: Isolar as "Fraudes" (Classe 1 - Causa + Sintomas)
    # =========================================================================
    df_fraudes_all = df_transacoes.filter(
        F.col("is_fraud") == 1
    ) # SEM .cache()
    
    count_fraudes_all = df_fraudes_all.count()

    if count_fraudes_all == 0:
        raise ValueError("ERRO CRÍTICO: Nenhuma transação com 'is_fraud = 1' foi encontrada.")
    
    print(f"INFO: Total de 'Fraudes' (Classe 1 - Original): {count_fraudes_all} linhas.")

    # =========================================================================
    # PASSO 2: Isolar as "Legítimas" (Classe 0)
    # =========================================================================
    
    df_legitimas_all = df_transacoes.filter(
        F.col("is_fraud") == 0
    ) # SEM .cache()
        
    count_legitimas_all = df_legitimas_all.count()
    
    if count_legitimas_all == 0:
        raise ValueError("ERRO CRÍTICO: Nenhuma transação com 'is_fraud = 0' foi encontrada.")
        
    print(f"INFO: Total de 'Legítimas' (Classe 0): {count_legitimas_all} linhas.")

    # =========================================================================
    # PASSO 3: APLICAR OVERSAMPLING (Sobreamostragem) no bloco de 'Fraudes'
    # =========================================================================
    print("INFO: Iniciando Etapa 1: Oversampling Multiclasse (Balanceando tipos de fraude)...")

    # 3a. Encontrar a contagem da classe de fraude majoritária
    df_counts = df_fraudes_all.groupBy("fraud_type").count()
    
    # Coletar a contagem máxima para o driver
    target_count_multiclass = df_counts.select(F.max("count")).first()[0]
    
    print(f"INFO: Classe de fraude majoritária tem {target_count_multiclass} amostras. Este é o alvo.")
    print("Distribuição de Fraudes (Antes do Oversampling):")
    df_counts.show()

    # 3b. Calcular o fator de repetição para cada classe
    df_fraudes_com_fator = df_fraudes_all.join(
        df_counts.withColumn("target_count", F.lit(target_count_multiclass)),
        "fraud_type"
    ).withColumn(
        "repeat_n", 
        F.ceil(F.col("target_count") / F.col("count")).cast("int") # Fator de repetição
    )

    # 3c. Explodir para duplicar as linhas das classes minoritárias
    df_oversampled_base = df_fraudes_com_fator.withColumn(
        "copy_index", 
        F.explode(F.sequence(F.lit(1), F.col("repeat_n")))
    )

    # 3d. RE-GERAR IDs para evitar chaves duplicadas (APENAS PARA CÓPIAS)
    df_fraudes_oversampled_com_novos_ids = df_oversampled_base.withColumn(
        "id_original", F.col("id") # Salva o ID original para referência
    ).withColumn(
        "id",
        F.when(
            F.col("copy_index") > 1, F.expr("uuid()") # Gera novo ID para cópias
        ).otherwise(F.col("id")) # Mantém o ID original
    ).withColumn(
        "id_transacao_cadeia_pai",
        F.when(
            F.col("copy_index") > 1, F.lit(None) # Cópias não são "filhas"
        ).otherwise(F.col("id_transacao_cadeia_pai")) # Mantém o pai original
    ).drop("count", "target_count", "repeat_n", "copy_index", "id_original")
    
    
    # 3e. Truncar (sample) de volta ao 'target_count'
    window_spec = Window.partitionBy("fraud_type").orderBy(F.rand(seed=SEED_BALANCEAMENTO))
    
    df_fraudes_balanceadas_multiclass = df_fraudes_oversampled_com_novos_ids \
        .withColumn("rank", F.row_number().over(window_spec)) \
        .filter(F.col("rank") <= target_count_multiclass) \
        .drop("rank") # SEM .cache()

    count_fraudes_balanceadas = df_fraudes_balanceadas_multiclass.count()
    print(f"INFO: Fraudes após oversampling (Etapa 1): {count_fraudes_balanceadas} linhas.")
    
    # =========================================================================
    # --- INÍCIO DA CORREÇÃO (PASSO 4 V11) ---
    # PASSO 4: BALANCEAMENTO BINÁRIO ROBUSTO (Undersampling da Classe Majoritária)
    # =========================================================================
    print("INFO: Iniciando Etapa 2: Balanceamento Binário (Identificando a classe majoritária)...")
    
    # count_fraudes_balanceadas (do PASSO 3)
    # count_legitimas_all (do PASSO 2)

    if count_fraudes_balanceadas == count_legitimas_all:
        print(f"INFO: Classes já estão balanceadas (1:1) com {count_fraudes_balanceadas} amostras cada. Nenhuma amostragem binária necessária.")
        df_fraudes_amostradas = df_fraudes_balanceadas_multiclass
        df_legitimas_amostradas = df_legitimas_all
    
    elif count_fraudes_balanceadas > count_legitimas_all:
        # CASO 1: Fraude é a MAIORIA (Aplicar Undersampling nas Fraudes)
        # Isso acontece em datasets de baixa escala onde o oversampling de fraudes supera as legítimas.
        print(f"INFO: Classe 'Fraude' é a majoritária ({count_fraudes_balanceadas} vs {count_legitimas_all}).")
        print("INFO: Aplicando UNDERSAMPLING na classe 'Fraude'...")
        
        fraction = count_legitimas_all / count_fraudes_balanceadas # (Será < 1.0)
        print(f"INFO: Fração de amostragem (Fraude): {fraction:.4f}")
        
        df_fraudes_amostradas = df_fraudes_balanceadas_multiclass.sample(
            withReplacement=False, 
            fraction=fraction, 
            seed=SEED_BALANCEAMENTO
        )
        df_legitimas_amostradas = df_legitimas_all # Manter todas as legítimas

    else:
        # CASO 2: Legítima é a MAIORIA (Aplicar Undersampling nas Legítimas)
        # Este é o cenário esperado em produção.
        print(f"INFO: Classe 'Legítima' é a majoritária ({count_legitimas_all} vs {count_fraudes_balanceadas}).")
        print("INFO: Aplicando UNDERSAMPLING na classe 'Legítima'...")
        
        fraction = count_fraudes_balanceadas / count_legitimas_all # (Será < 1.0)
        print(f"INFO: Fração de amostragem (Legítima): {fraction:.4f}")
        
        df_legitimas_amostradas = df_legitimas_all.sample(
            withReplacement=False, 
            fraction=fraction, 
            seed=SEED_BALANCEAMENTO
        )
        df_fraudes_amostradas = df_fraudes_balanceadas_multiclass # Manter todas as fraudes
    


 
    # PASSO 5: Unir e Salvar o Dataset Final

    df_transacoes_balanced = df_fraudes_amostradas.unionByName(df_legitimas_amostradas)

    print("INFO: Materializando dataset balanceado (V11-Robusto) como 'transacoes_db.gold.transacoes_balanced_model'...")
    spark.sql("CREATE SCHEMA IF NOT EXISTS transacoes_db.gold")
    
    df_transacoes_balanced.write \
        .mode("overwrite") \
        .format("delta") \
        .saveAsTable("transacoes_db.gold.transacoes_balanced_model")
    
    print("✅ SUCESSO: Dataset balanceado (V11-Robusto) salvo.")

finally:
    # Bloco 'finally' agora está seguro e não tenta limpar caches
    print("INFO: Processo finalizado (sem caches para limpar).")


# =============================================================================
# CÉLULA 6 (Verificação) - Execute esta após a Célula 5
# =============================================================================

print("\n\n--- VERIFICAÇÃO PÓS-BALANCEAMENTO (V11-ROBUSTO) ---")
df_check = spark.table("transacoes_db.gold.transacoes_balanced_model")


total_count_balanced = df_check.count()
print(f"Total de Registros (Balanceado): {total_count_balanced}")

print("\nDistribuição por Classe (Balanceado):")
df_check.groupBy("is_fraud").count().show()
# Esperado: 50/50 (ou muito próximo)

print("\nDistribuição por Tipo de Fraude (Balanceado):")
df_check.withColumn(
        "tipo_fraude_detalhado", 
        F.when(F.col("is_fraud") == 0, "Legitima").otherwise(F.col("fraud_type"))
    ) \
    .groupBy("tipo_fraude_detalhado") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show()
# Esperado: 'Legitima' (50%) e todos os tipos de fraude
# (ex: 'valor_atipico', 'eng_social', etc.) com contagens IDÊNTICAS.

## Analise visual Pos Balanceamento

In [0]:
import matplotlib.pyplot as plt

# Carregar dados balanceados
df_check = spark.table("transacoes_db.gold.transacoes_balanced_model")

# Coletar distribuição por classe
dist_classe = df_check.groupBy("is_fraud").count().orderBy("is_fraud")
classe_pd = dist_classe.toPandas()

# Gráfico de barras: Distribuição por Classe
plt.figure(figsize=(6,4))
plt.bar(classe_pd['is_fraud'], classe_pd['count'], color=['#2ca02c', '#d62728'])
plt.xticks([0,1], ['Legítima', 'Fraude'])
plt.title('Distribuição por Classe (Balanceado)')
plt.xlabel('Classe')
plt.ylabel('Contagem')
plt.tight_layout()
plt.show()



In [0]:
# Coletar distribuição por tipo de fraude
df_tipo = df_check.withColumn(
    "tipo_fraude_detalhado", 
    F.when(F.col("is_fraud") == 0, "Legitima").otherwise(F.col("fraud_type"))
).groupBy("tipo_fraude_detalhado").count().orderBy(F.col("count").desc())
tipo_pd = df_tipo.toPandas()

# Gráfico de barras: Distribuição por Tipo de Fraude
plt.figure(figsize=(10,5))
plt.bar(tipo_pd['tipo_fraude_detalhado'], tipo_pd['count'], color='#1f77b4')
plt.title('Distribuição por Tipo de Fraude (Balanceado)')
plt.xlabel('Tipo de Fraude')
plt.ylabel('Contagem')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [0]:
# Histograma de valores transacionados por classe (Pós Balanceamento)
valores_pd = df_check.select(
    "is_fraud", 
    "valor"
).toPandas()

plt.figure(figsize=(8,5))
for label, color in zip([0,1], ['#2ca02c', '#d62728']):
    plt.hist(
        valores_pd[valores_pd['is_fraud']==label]['valor'], 
        bins=30, 
        alpha=0.6, 
        label=f'{"Legítima" if label==0 else "Fraude"}', 
        color=color
    )
plt.title('Distribuição dos Valores Transacionados por Classe (Pós Balanceamento)')
plt.xlabel('Valor da Transação')
plt.ylabel('Frequência')
plt.legend()
plt.tight_layout()
plt.show()